In [ ]:
import requests
from bs4 import BeautifulSoup
# 多线程执行
import threading
"""
    author:李梓瑀
    date:2024-03-08
"""
# 待爬取信息:唐诗三百首,类型,诗歌作者,年代,诗歌本体
# 修改全局默认编码方式为 UTF-8
requests.models.Response.encoding = 'utf-8'
# 设置路径字典
urlDict = {
    "baseUrl":"https://so.gushiwen.cn",
    # 子路由
    "childUrl":{
        # 诗文
        "shiwens":"/shiwens",
        # 名句
        "mingjus":"/mingjus",
        # 作者
        "authors":"/authors",
        # 古文
        "guwen":"/guwen",
        # 古诗
        "gushi":{
            # 唐诗三百首
            "tangshi":"/gushi/tangshi.aspx",
            #古诗三百首
            "gushi":"/gushi/sanbai.aspx"
        },
        # 我的
        "my":"/user/collect.aspx"
    }
}
# 设置请求头信息
myHeaders = {
    # 设置cookie
    "Cookie":"gswZhanghao=15042070108; login=flase; b-user-id=8912c27c-b8b1-3ea1-e04c-96ad0cb19798; ticketStr=208361286%7cgQE48DwAAAAAAAAAAS5odHRwOi8vd2VpeGluLnFxLmNvbS9xLzAyTzBmTFJDbGVkN2kxZFMxY05DMXAAAgR2NOVlAwQAjScA; Hm_lvt_9007fab6814e892d3020a64454da5a55=1709518422,1709861171; gsw2017user=5723808%7c6A5471B38CFFFF27880E4F7E9679CF7Aec6584b1%7c2000%2f1%2f1%7c2000%2f1%2f1; login=flase; wxopenid=oVc5H0nj-AeE99803X475zDXAWuM; Hm_lpvt_9007fab6814e892d3020a64454da5a55=1709862947",
    # 设置UA
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
}
# 设置诗歌数据列表
dataList = []
# 设置线程池
threads = []
# 设置诗歌类
class Poetry:
    # 诗歌名
    name:str = None
    # 诗歌作者
    author:str = None
    # 作者朝代
    dynasty:str = None
    # 诗歌本体
    body:str = None
    # 参考资料
    info:str = None
    def __init__(self,name,author,dynasty,body,info):
        self.name = name
        self.author = author
        self.dynasty = dynasty
        self.body = body
        self.info = info

In [3]:
#爬取数据
# 获取歌曲信息
def getInfo(url:str,type:str):
    global dataList
    detail_html = requests.get(url=url,headers=myHeaders).text
    bs_detail = BeautifulSoup(detail_html)
    # 诗歌详情
    shige_details = bs_detail.select_one("#sonsyuanwen>.cont")
    # 诗歌译文及注释
    shige_translation = bs_detail.select_one(".cankao>div>span:nth-child(2)")
    author = shige_details.select_one(".source>a").text.strip()
    # 作者朝代
    dynasty = shige_details.select_one(".source a:nth-child(2)").text.strip()[1:-1]
    # 诗歌原文
    body = shige_details.select_one(".contson").text.strip()
    # 诗歌参考资料
    info = "暂无参考资料" if shige_translation is None else shige_translation.text.split("：")[0].strip()
    myPoetry = Poetry(type,author,dynasty,body,info)
    dataList.append(myPoetry)
# 唐诗三百首
def getTangshi():
    # 线程池
    global threads
    tangshi_url = urlDict['baseUrl']+urlDict['childUrl']['gushi']['tangshi']
    sanbai_html = requests.get(url=tangshi_url,headers=myHeaders).text
    #解析主页,按照类型
    bs_tangshiByType = BeautifulSoup(sanbai_html).select(".main3>.left>.sons>.typecont")
    #唐诗三百首解析
    for poetry in bs_tangshiByType:
        # 唐诗类型
        type = poetry.select_one(".bookMl>strong").text
        for detail in poetry.select("span"):
            newurl = urlDict['baseUrl']+detail.select_one("a").get("href")
            # 创建一个线程,并执行getinfo方法
            thread = threading.Thread(target=getInfo,args=(newurl,type))
            thread.start()
            # 加入线程池
            threads.append(thread)
        for thread in threads:
            # 暂时阻塞
            thread.join()
# 古诗合集
# def getGUshi():
getTangshi()

In [4]:
import openpyxl
import os
# 使用openpyxl存储数据
# 新建工作簿
path = r"../数据集/诗歌数据.xlsx"
# 没有则新建,有则获取到sheet工作表
titleHeader = ["诗歌名称","诗歌作者","作者朝代","诗歌原文","参考资料"]
# 如果有则
if not os.path.exists(path):
    wb = openpyxl.Workbook()
    wb.save(path)
else:
    wb = openpyxl.load_workbook(path)
sheet = wb['Sheet']
# 设置单元格标题
if sheet.cell(1,1).value != titleHeader[0]:
    for i in range(1,len(titleHeader)+1):
        sheet.cell(1,i).value = titleHeader[i-1]
    wb.save(path)
# 存入数据
for data in dataList:
    # 对象解构为列表
    sheet.append(list(vars(data).values()))
# 设置列宽为80
sheet.column_dimensions['D'].width = 80
sheet.column_dimensions['E'].width = 80
wb.save(path)